In [8]:
from typing import Optional, Dict, List
import json

In [9]:
class ContextObject:
    def __init__(
        self, 
        role: Optional[str] = None, 
        content: Optional[str] = None, 
        name: Optional[str] = None,
        status: Optional[str] = None,      # 仅用于控制可见性/可编辑性
        model_tags: Optional[List[str]] = None,  # 新增：模型能力标签
        metadata: Optional[Dict] = None,
        id: Optional[str] = None,
        parent_id: Optional[str] = None
    ):
        self.role = role
        self.content = content
        self.name = name
        self.status = status
        self.model_tags = model_tags or []  # 默认空列表
        self.metadata = metadata or {}
        self.id = id
        self.parent_id = parent_id

    def to_dict(self) -> Dict:
        return {
            "role": self.role,
            "content": self.content,
            "name": self.name,
            "status": self.status,
            "model_tags": self.model_tags,  # 序列化新增字段
            "metadata": self.metadata,
            "id": self.id,
            "parent_id": self.parent_id
        }

    @classmethod
    def from_dict(cls, data: Dict) -> 'ContextObject':
        return cls(
            role=data.get("role"),
            content=data.get("content"),
            name=data.get("name"),
            status=data.get("status"),
            model_tags=data.get("model_tags", []),  # 反序列化新增字段
            metadata=data.get("metadata", {}),
            id=data.get("id"),
            parent_id=data.get("parent_id")
        )

In [10]:
class PromptUnit:

    def __init__(
            self,
            role: str,
            content: str,
            status: Optional[str] = None,
            metadata: Optional[Dict] = None,
            id: Optional[str] = None,    
        ):
        self.role = role # user, assistant, system, tool, tool_response
        self.content = content # can be text, image, audio, video, file, etc.
        self.status = status # locked, hidden, readonly, etc.
        self.metadata = metadata or {} # 附加边界控制信息
        self.id = id
    
    def is_visible(self) -> bool:
        return self.status not in ['hidden', 'deleted']

    def is_editable(self) -> bool:
        return self.status not in ['locked', 'readonly', 'deleted']
    
    def to_dict(self) -> Dict:
        return {
            'role': self.role,
            'content': self.content,
            'status': self.status,
            'metadata': self.metadata,
            'id': self.id,
        }

In [11]:
# 上下文链
class ContextChain:
    """
    组织ContextObject的链式结构，支持多轮对话和复杂交互
    作用：
        1. 多轮对话语境重建：通过parent_id追踪对话历史，重建完整语境链条
        2. 任务分解与Agent协作：不同角色的ContextObject可以分布在链条不同位置，支持复杂任务分解和Agent协作
        3. 错误追踪与调试：通过链式结构可以清晰追踪上下文流转，定位问题根源，回退到特定节点进行调整
        4. 语义路径审计与响应追踪：链式结构便于审计语义路径，追踪响应生成过程，确保透明可控
    """
    def __init__(self):
        self.nodes: Dict[str, ContextObject] = {} # 存储ContextObject的字典

    def add_prompt(self, prompt: ContextObject):
        if not prompt.id:
            raise ValueError("ContextObject必须有唯一的id")
        self.nodes[prompt.id] = prompt

    def trace_chain(self, start_id: str) -> List[ContextObject]:
        """
        追踪从start_id开始的上下文链，返回链上所有相关对象
        """
        chain = []
        current_node = self.nodes.get(start_id)
        while current_node:
            chain.insert(0, current_node)  # 插入到链表开头，保持顺序
            if not current_node.parent_id:
                break
            current_node = self.nodes.get(current_node.parent_id)
        return chain
    
    def create_snapshot(self, end_id: str) -> str:
        """
        创建从end_id开始的上下文链快照，返回链上所有相关对象的字典表示
        """
        chain = self.trace_chain(end_id)
        data = [node.to_dict() for node in chain]
        return json.dumps(data, indent=2, ensure_ascii=False)
    
    def restore_from_snapshot(self, snapshot: str):
        self.nodes.clear()
        snapshot_data = json.loads(snapshot)
        
        # 防护空快照
        if not snapshot_data:
            return []
        
        for node_data in snapshot_data:
            context_object = ContextObject.from_dict(node_data)
            self.add_prompt(context_object)
        
        # 安全获取末端节点ID
        end_id = snapshot_data[-1].get('id')
        return self.trace_chain(end_id) if end_id else []

In [12]:
# 共享上下文
class SharedContext:

    def __init__(self, context_id: str, prompts: List[ContextObject]):
        self.context_id = context_id
        self.prompts = prompts # 共享的ContextObject列表

    def filter_for_model(self, model_capability: str) -> List[ContextObject]:
        """
        保留适用于指定模型能力的 prompts
        - 无 model_tags 的对象默认对所有模型可见
        - 有 model_tags 的对象需精确匹配
        """
        return [
            prompt for prompt in self.prompts 
            if not prompt.model_tags or model_capability in prompt.model_tags
        ]

    def to_model_input(self) -> List[Dict]:
        return [prompt.to_dict() for prompt in self.prompts]
    

# 修复拼写错误 + 使用 model_tags 替代 status
ctx = SharedContext(
    context_id="ctx1",
    prompts=[
        ContextObject(
            role="user", 
            content="请帮我写一首诗", 
            id="p1", 
            model_tags=["code_model"]  # 修复：使用 model_tags
        ),
        ContextObject(
            role="assistant", 
            content="好的，请问你喜欢什么主题的诗？", 
            id="p2", 
            parent_id="p1",
            model_tags=["code_model"]
        ),
        ContextObject(
            role="user", 
            content="我喜欢自然风光的诗", 
            id="p3", 
            parent_id="p2",
            model_tags=["general_model"]
        ),
    ]
)

# 修复变量名拼写
code_model_input = ctx.filter_for_model("code_model")  # 原 codel_model_input
print([p.to_dict() for p in code_model_input])

general_model_input = ctx.filter_for_model("general_model")
print([p.to_dict() for p in general_model_input])

[{'role': 'user', 'content': '请帮我写一首诗', 'name': None, 'status': None, 'model_tags': ['code_model'], 'metadata': {}, 'id': 'p1', 'parent_id': None}, {'role': 'assistant', 'content': '好的，请问你喜欢什么主题的诗？', 'name': None, 'status': None, 'model_tags': ['code_model'], 'metadata': {}, 'id': 'p2', 'parent_id': 'p1'}]
[{'role': 'user', 'content': '我喜欢自然风光的诗', 'name': None, 'status': None, 'model_tags': ['general_model'], 'metadata': {}, 'id': 'p3', 'parent_id': 'p2'}]
